#  Automated Grid Search (With 1000 Trials)

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split, GridSearchCV

tk = 'AAPL'
ticker = yf.Ticker(tk)
df = ticker.history(interval='1d', start='2022-01-01')

df = df['Close'].values
df = df.reshape(-1, 1)

dataset_train = np.array(df[:int(df.shape[0]*0.8)])
dataset_test = np.array(df[int(df.shape[0]*0.8):])

scaler = MinMaxScaler(feature_range=(0,1))
dataset_train = scaler.fit_transform(dataset_train)
dataset_test = scaler.transform(dataset_test)

def create_dataset(df):
    x = []
    y = []
    for i in range(50, df.shape[0]):
        x.append(df[i-50:i, 0])
        y.append(df[i, 0])
    x = np.array(x)
    y = np.array(y)
    return x,y


x_train, y_train = create_dataset(dataset_train)
x_test, y_test = create_dataset(dataset_test)


x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))


# Defining the function to build the model
def build_lstm_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units_layer1', min_value=100, max_value=500, step=50),
                   return_sequences=True, input_shape=(x_train.shape[1], 1)))
    model.add(Dropout(hp.Float('dropout_rate_layer1', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units_layer2', min_value=100, max_value=500, step=50),
                   return_sequences=True))
    model.add(Dropout(hp.Float('dropout_rate_layer2', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units_layer3', min_value=100, max_value=500, step=50),
                   return_sequences=True))
    model.add(Dropout(hp.Float('dropout_rate_layer3', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units_layer4', min_value=100, max_value=500, step=50)))
    model.add(Dropout(hp.Float('dropout_rate_layer4', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(units=1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# Instantiating the RandomSearch tuner (Creating tuner object)
tuner = RandomSearch(
    build_lstm_model,
    objective='val_loss',
    max_trials=1500,
    executions_per_trial=1,
    directory='my_dir',  # Specify a directory to save results
    project_name='my_project'
)

# Searching for the best hyperparameters
tuner.search(x_train, y_train, validation_data=(x_test, y_test))

# Inside the loop that trains the models
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:")
print("- units_layer1:", best_hps.get('units_layer1'))
print("- dropout_rate_layer1:", best_hps.get('dropout_rate_layer1'))
print("- units_layer2:", best_hps.get('units_layer2'))
print("- dropout_rate_layer2:", best_hps.get('dropout_rate_layer2'))
print("- units_layer3:", best_hps.get('units_layer3'))
print("- dropout_rate_layer3:", best_hps.get('dropout_rate_layer3'))
print("- units_layer4:", best_hps.get('units_layer4'))
print("- dropout_rate_layer4:", best_hps.get('dropout_rate_layer4'))

Trial 1500 Complete [00h 00m 46s]
val_loss: 0.02931946888566017

Best val_loss So Far: 0.003012970555573702
Total elapsed time: 00h 10m 09s
INFO:tensorflow:Oracle triggered exit
Best hyperparameters:
- units_layer1: 500
- dropout_rate_layer1: 0.4
- units_layer2: 500
- dropout_rate_layer2: 0.30000000000000004
- units_layer3: 500
- dropout_rate_layer3: 0.2
- units_layer4: 200
- dropout_rate_layer4: 0.4


# Combined Hyperparameter Tuning for multiple stocks

In [2]:
import numpy as np
import yfinance as yf 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras_tuner.tuners import RandomSearch

# Define a list of stock tickers
stock_tickers = ['AAPL', 'MSFT', 'ORCL']  # Add more tickers as needed

# Create an empty array to store combined data
combined_data = []

# Loop through the stock tickers to fetch and preprocess data
for tk in stock_tickers:
    ticker = yf.Ticker(tk)
    df = ticker.history(interval='1d', start='2022-01-01', end='2023-09-11')
    df = df['Close'].values
    df = df.reshape(-1, 1)
    combined_data.append(df)

combined_data = np.concatenate(combined_data, axis=1)  # Combine data

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(combined_data)

def create_dataset(df):
    x = []
    y = []
    for i in range(50, df.shape[0]):
        x.append(df[i-50:i, :])
        y.append(df[i, :])
    x = np.array(x)
    y = np.array(y)
    return x, y

x_train, y_train = create_dataset(scaled_data)

x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], x_train.shape[2]))

def build_lstm_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units_layer1', min_value=100, max_value=500, step=50),
                   return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(Dropout(hp.Float('dropout_rate_layer1', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units_layer2', min_value=100, max_value=500, step=50),
                   return_sequences=True))
    model.add(Dropout(hp.Float('dropout_rate_layer2', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units_layer3', min_value=100, max_value=500, step=50),
                   return_sequences=True))
    model.add(Dropout(hp.Float('dropout_rate_layer3', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units_layer4', min_value=100, max_value=500, step=50)))
    model.add(Dropout(hp.Float('dropout_rate_layer4', min_value=0.2, max_value=0.5, step=0.1)))
    model.add(Dense(units=x_train.shape[2]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

tuner = RandomSearch(
    build_lstm_model,
    objective='val_loss',
    max_trials=8000,
    executions_per_trial=1,
    directory='my_dir_2',  # Specify a directory to save results
    project_name='my_project_2'
)

tuner.search(x_train, y_train, validation_split=0.2)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:")
print("- units_layer1:", best_hps.get('units_layer1'))
print("- dropout_rate_layer1:", best_hps.get('dropout_rate_layer1'))
print("- units_layer2:", best_hps.get('units_layer2'))
print("- dropout_rate_layer2:", best_hps.get('dropout_rate_layer2'))
print("- units_layer3:", best_hps.get('units_layer3'))
print("- dropout_rate_layer3:", best_hps.get('dropout_rate_layer3'))
print("- units_layer4:", best_hps.get('units_layer4'))
print("- dropout_rate_layer4:", best_hps.get('dropout_rate_layer4'))

INFO:tensorflow:Reloading Tuner from my_dir_2\my_project_2\tuner0.json


RuntimeError: Error reloading `Oracle` from existing project. If you did not mean to reload from an existing project, change the `project_name` or pass `overwrite=True` when creating the `Tuner`. Found existing project at: my_dir_2\my_project_2

# For multiple stock tickers

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split, GridSearchCV

# List of ticker symbols
ticker_symbols = ['AAPL', 'MSFT', 'ORCL']  # Add more ticker symbols as needed

# Define a function to perform hyperparameter tuning for a single ticker
def tune_hyperparameters_for_ticker(tk):
    # Fetch data for the ticker
    ticker = yf.Ticker(tk)
    df = ticker.history(interval='1d', start='2022-01-01')
    
    df = df['Close'].values
    df = df.reshape(-1, 1)

    dataset_train = np.array(df[:int(df.shape[0]*0.8)])
    dataset_test = np.array(df[int(df.shape[0]*0.8):])

    scaler = MinMaxScaler(feature_range=(0,1))
    dataset_train = scaler.fit_transform(dataset_train)
    dataset_test = scaler.transform(dataset_test)

    x_train, y_train = create_dataset(dataset_train)
    x_test, y_test = create_dataset(dataset_test)

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    # Define the function to build the model as you did before
    def build_lstm_model(hp):
        model = Sequential()
        model.add(LSTM(units=hp.Int('units_layer1', min_value=100, max_value=500, step=50),
                       return_sequences=True, input_shape=(x_train.shape[1], 1)))
        model.add(Dropout(hp.Float('dropout_rate_layer1', min_value=0.2, max_value=0.5, step=0.1)))
        model.add(LSTM(units=hp.Int('units_layer2', min_value=100, max_value=500, step=50),
                       return_sequences=True))
        model.add(Dropout(hp.Float('dropout_rate_layer2', min_value=0.2, max_value=0.5, step=0.1)))
        model.add(LSTM(units=hp.Int('units_layer3', min_value=100, max_value=500, step=50),
                       return_sequences=True))
        model.add(Dropout(hp.Float('dropout_rate_layer3', min_value=0.2, max_value=0.5, step=0.1)))
        model.add(LSTM(units=hp.Int('units_layer4', min_value=100, max_value=500, step=50)))
        model.add(Dropout(hp.Float('dropout_rate_layer4', min_value=0.2, max_value=0.5, step=0.1)))
        model.add(Dense(units=1))
        model.compile(loss='mean_squared_error', optimizer='adam')
        return model

    # Instantiate the RandomSearch tuner
    tuner = RandomSearch(
        build_lstm_model,
        objective='val_loss',
        max_trials=1500,
        executions_per_trial=1,
        directory='my_dir',  # Specify a directory to save results
        project_name='my_project'
    )

    # Search for the best hyperparameters
    tuner.search(x_train, y_train, validation_data=(x_test, y_test))

    # Get the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print(f"Best hyperparameters for {tk}:")
    print("- units_layer1:", best_hps.get('units_layer1'))
    print("- dropout_rate_layer1:", best_hps.get('dropout_rate_layer1'))
    print("- units_layer2:", best_hps.get('units_layer2'))
    print("- dropout_rate_layer2:", best_hps.get('dropout_rate_layer2'))
    print("- units_layer3:", best_hps.get('units_layer3'))
    print("- dropout_rate_layer3:", best_hps.get('dropout_rate_layer3'))
    print("- units_layer4:", best_hps.get('units_layer4'))
    print("- dropout_rate_layer4:", best_hps.get('dropout_rate_layer4'))

# Iterate over the list of ticker symbols and perform hyperparameter tuning for each ticker
for tk in ticker_symbols:
    tune_hyperparameters_for_ticker(tk)

NameError: name 'create_dataset' is not defined